In [1]:
import pandas as pd
import numpy as np
import glob

Okay we've downloaded the files we need from [this dataset](https://www.kaggle.com/cclayford/cricinfo-statsguru-data/data). I just run these bash commands to rename the files for easier shell handling <br> 

find *.csv -exec rename 's/\s/_/g' {} \;

Then to make all the files lowercase:
find *.csv -exec rename 'y/A-Z/a-z/' {} \;

To fix up the headings, we do the same for the first line in all the csv files.

1s/.*/\L&/ < on the first line, make sure all characters are lowercase \L
1s/ /_/g ... on the first line, replace all spaces with underscores, globally

find *.csv -exec sed -i '1s/.*/\L&/;1s/ /_&/' {} \;

Okay that's it. We've done the file processing, now to load them all into jupyter. 

I'm just going to be looking at the Men's test individual data. But you could do the same for the women, ODIs, etc. Just load those into the dataframe instead.

In [2]:
all_files = glob.glob('data/men_test_player_innings_stats_-_*.csv')

# low_memory=False just tries to figure out the data type of large files.
# we don't really need this, because we'll fix these later.
df = pd.concat((pd.read_csv(f, low_memory=False) for f in all_files))


There's a few things we should do before we can start processing the data.
- drop the duplicate rows
- change the type of data to the appropriate data type in each column (I *definitely* knew about this before starting the post, yep, *definitely*)
- Sort by date - which will be a handy default

In [3]:
df = df.drop_duplicates()

int_cols = ['innings_runs_scored_num', 'innings_minutes_batted', 
            'innings_batted_flag', 'innings_not_out_flag', 'innings_bowled_flag',
            'innings_balls_faced', 'innings_boundary_fours',
            'innings_boundary_sixes', 'innings_maidens_bowled',
            'innings_runs_conceded', 'innings_wickets_taken',
            '4_wickets', '5_wickets', '10_wickets'
           ]

for col in int_cols:
    # use to_numeric to convery the - to NaN (not-a-number)
    # "Int32" has mixed NaNs and integers
    df[col] = pd.to_numeric(df[col], errors='coerce').astype("Int32")
    

df.innings_economy_rate = pd.to_numeric(df.innings_economy_rate, errors='coerce')
df.innings_date = pd.to_datetime(df.innings_date, infer_datetime_format=True)
df = df.sort_values(by=['innings_date'])

Okay, so now we've gotten to the point where we hoped we would have started, clean data, I guess. How do we know we haven't fucked something up? 

Well, Don Bradman's test batting average was 99.94, famously, how about we write a function to check the average is correct. 

SQL is feeling a little bit lame, honestly, so how about we stick to what we know and write a function to get the average.

Ao the average of a batter is the number of runs scored in their career minus the number of times they have been dismissed.

So we can write a function for each.

In [4]:
def get_total_runs(player):
    return df.loc[df['innings_player'] == player, 'innings_runs_scored_num'].sum()

def times_dismissed(player):
    num_innings = len(df.loc[df['innings_player'] == player, 'innings_runs_scored_num'].dropna())
    not_out = df.loc[df['innings_player'] == player, 'innings_not_out_flag'].sum()
    return num_innings - not_out

def get_average(player):
    runs = get_total_runs(player)
    dismissed = times_dismissed(player)
    if dismissed == 0:
        return np.nan
    return runs / dismissed
print(get_average('DG Bradman'))

99.94285714285714


Fuckn yeh boooooiiiii!

That was fun an all, but really, what we want is to do some fucking number crunching here. We can calculate the average of every player in the history of the game.

First we want to get all the players...

In [5]:
all_players = pd.DataFrame(df.innings_player.drop_duplicates())

averages = all_players # make a copy because we'll use all_players a lot later.
averages['average'] = averages.innings_player.apply(get_average)

# filter out infinite and not-a-number results
averages.replace([np.inf, -np.inf], np.nan, inplace=True)
averages.dropna(inplace=True)
averages.sort_values(by=['average'], ascending=False, inplace=True)
print(averages[:10])

      innings_player     average
22755   KR Patterson  144.000000
66931   AG Ganteaume  112.000000
48719       Abid Ali  107.000000
50574     DG Bradman   99.942857
62077       MN Nawaz   99.000000
60884  VH Stollmeyer   96.000000
85275       DM Lewis   86.333333
69175     Abul Hasan   82.500000
91237     RE Redmond   81.500000
30714    DJ Mitchell   73.000000


What the fuck! That can't be real

In [6]:

def get_all_player_stats(player):
    # return the entire dataframe but filtered for that player
    return df.loc[df['innings_player'] == player]

def num_ducks(player):
    all_stats = get_all_player_stats(player)
    ducks = all_stats[(all_stats.innings_not_out_flag == 0) & (all_stats.innings_runs_scored_num == 0)]
    return len(ducks)
print(f"The Don got {num_ducks('DG Bradman')} ducks")

The Don got 7 ducks


In [7]:
#pd.options.mode.chained_assignment = None

def get_all_player_stats(player, data):
    # return the entire dataframe but filtered for that player
    return data[data['innings_player'] == player]

def get_all_teams_stats(team, data):
    return data[data['team'] == team]
    

def num_matches(test, col, test_num, df_data, greater, individual=True):
    # https://stackoverflow.com/questions/40068261/pandas-dataframe-find-longest-consecutive-rows-with-a-certain-condition
    # Special mention to the above post for this solution.
    data = df_data.copy()
    data = data[data[col] != np.nan]
    if greater:
        data['condition_true'] = np.where(data[col] > test_num, True, False)
    else:
        data['condition_true'] = np.where(data[col] < test_num, True, False)
    if individual:
        all_stats = get_all_player_stats(test, data)
    else:
        all_stats = get_all_teams_stats(test, data)
    df_bool = all_stats['condition_true'] != all_stats['condition_true'].shift()
    df_cumsum = df_bool.cumsum()
    groups = all_stats.groupby(df_cumsum)
    group_counts = groups.agg({col: ['count', 'min', 'max']})
    group_counts.columns = group_counts.columns.droplevel()
    if greater:
        group_counts = group_counts[group_counts['min'] > test_num]
    else:
        group_counts = group_counts[group_counts['max'] < test_num]
    max_count = group_counts['count'].max()
    return max_count

def num_matches_group(test, col, test_num, df_data, greater, individual=True):
    # https://stackoverflow.com/questions/40068261/pandas-dataframe-find-longest-consecutive-rows-with-a-certain-condition
    # Special mention to the above post for this solution.
    data = df_data.copy()
    data = data[data[col] != np.nan]
    if greater:
        data['condition_true'] = np.where(data[col] > test_num, True, False)
    else:
        data['condition_true'] = np.where(data[col] < test_num, True, False)
#     print(data)
    if individual:
        all_stats = get_all_player_stats(test, data)
    else:
        all_stats = get_all_teams_stats(test, data)
    df_bool = all_stats['condition_true'] != all_stats['condition_true'].shift()
    df_cumsum = df_bool.cumsum()
    groups = all_stats.groupby(df_cumsum)
    
    # this is the worst code I've ever written.
    # please don't judge, it's 1am and I got it to work.
    # I'm sure groupby has a thing that does this, but I am too tired to figure it out.
    max_len = 0
    resulting_df = None
    for g in groups:
        if g[1].condition_true.all() == True:
            if len(g[1]) > max_len:
                max_len = len(g[1])
                resulting_df = g[1]
    return max_len, resulting_df


In [14]:

# Enter these values to match a condition
match_name = 'greater_40'
column = 'innings_runs_scored_num'
greater_than = True
condition_number = 40

# comment out one of these lines to match the dataset you want
match_data = df.copy()
# match_data = team_innings.copy()


all_players['greater_40'] = all_players.innings_player.apply(
    num_matches, 
    args=(column, condition_number, match_data, greater_than))
all_players.sort_values(by=['greater_40'], ascending=False, inplace=True)
print(all_players[:5])


      innings_player    average  greater_40
28904      JH Kallis  55.255230        10.0
84190      ED Weekes  58.618421         9.0
99630  Javed Miandad  52.571429         8.0
19156       SR Waugh  51.060748         8.0
57567      EJ Barlow  45.745455         8.0


In [26]:
top_player = all_players.iloc[0,0]

num, scores = num_matches_group(top_player, column, condition_number, df, greater_than, True)

cols_to_print = ['innings_player', 'innings_runs_scored', 'opposition' ,'ground', 'innings_date', 
                 'innings_minutes_batted', 'innings_balls_faced', 'innings_boundary_fours',
                 'innings_boundary_sixes','innings_batting_strike_rate', 'innings_number']
print(scores[cols_to_print])

      innings_player innings_runs_scored     opposition        ground  \
10166      JH Kallis                  43     v Pakistan    Faisalabad   
10151      JH Kallis                  44  v West Indies  Johannesburg   
9476       JH Kallis                 158  v West Indies  Johannesburg   
9459       JH Kallis                 177  v West Indies        Durban   
9536       JH Kallis                130*  v West Indies     Cape Town   
9784       JH Kallis                  73  v West Indies     Cape Town   
9537       JH Kallis                130*  v West Indies     Centurion   
9489       JH Kallis                150*  v New Zealand      Hamilton   
9679       JH Kallis                  92  v New Zealand      Hamilton   
9798       JH Kallis                  71  v New Zealand      Auckland   

      innings_date  innings_minutes_batted  innings_balls_faced  \
10166   2003-10-24                     174                  113   
10151   2003-12-12                      96                   7

So that lets us get all the innings that match. It's horrible code, I know, there has to be something that does it in one line. But I want to get this out the door working, and fix the computer-science-nerd-wank later.

Jacques Kallis is a boss, it turns out, 10 innings in a row greater than 40. What a boss.


So what if we wanted to perform the same action for a team innings. Well, the data we downloaded did not have all the indvidual team innings, unfortunately. But the data does exist on statsguru. So, I've modified a scraper that I found online to download the data. You can check out the [repo here](https://github.com/obrasier/statsguru-scraper)

In [32]:
all_innings_file = 'data/all_test_innings.csv'
all_innings = pd.read_csv(all_innings_file)
all_innings.start_date = pd.to_datetime(all_innings.start_date, infer_datetime_format=True)
all_innings.runs = pd.to_numeric(all_innings.runs, errors='coerce').astype("Int64")

all_teams = pd.DataFrame(all_innings.team.drop_duplicates())


# Enter these values to match a condition
match_name = 'under_200'
column = 'runs'
greater_than = False
condition_number = 200

# comment out one of these lines to match the dataset you want
# match_data = df.copy()
match_data = all_innings.copy()

# all_players['greater_40'] = all_players.innings_player.apply(
#     num_matches, 
#     args=(column, condition_number, all_innings, greater_than))
# all_players.sort_values(by=['greater_40'], ascending=False, inplace=True)
# print(all_players[:5])


all_teams[match_name] = all_teams.team.apply(
    num_matches, 
    args=(column, condition_number, all_innings, greater_than, False))
all_teams.sort_values(by=[match_name], ascending=False, inplace=True)

worst_team = all_teams.iloc[0,0]
print(worst_team)

amount, condition_innings = num_matches_group('Australia', 'runs', 200, all_innings, False, False)

columns_to_print = ['team', 'score', 'overs', 'opposition', 'ground', 'start_date']
print(amount)
print(condition_innings[columns_to_print])

Australia
21
          team score  overs opposition      ground start_date
80   Australia   123  118.3  v England  Manchester 1886-07-05
83   Australia   121   82.3  v England      Lord's 1886-07-19
84   Australia   126  111.1  v England      Lord's 1886-07-19
86   Australia    68   60.2  v England    The Oval 1886-08-12
87   Australia   149   97.0  v England    The Oval 1886-08-12
89   Australia   119  113.1  v England      Sydney 1887-01-28
91   Australia    97  107.0  v England      Sydney 1887-01-28
93   Australia    84   55.1  v England      Sydney 1887-02-25
95   Australia   150  110.0  v England      Sydney 1887-02-25
97   Australia    42   37.3  v England      Sydney 1888-02-10
99   Australia    82   69.2  v England      Sydney 1888-02-10
100  Australia   116   71.2  v England      Lord's 1888-07-16
102  Australia    60   29.2  v England      Lord's 1888-07-16
104  Australia    80   90.3  v England    The Oval 1888-08-13
106  Australia   100   69.2  v England    The Oval 1888-0

Okay so we got what we were after, but we want more! Like a dog who wants a bone, we can never get enough stats. 21 scores under 200 in a row! When the hell were they! The last piece of our puzzle is to get the rows that match, and give us back the original dataframe so we can see where Australia's terrible streak was. Let me tell you now, this is a hard problem.

So it works! We answered the queries, but it's a bit bloody clunky. I'm going to just submit this post now, but I will edit it. Because I'm sure I should have been using [df.query](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) this whole time to make it easier to write generic queries instead of the filtering I've been doing. 